In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q git+https://github.com/huggingface/accelerate.git@main
!pip install bitsandbytes SentencePiece

In [1]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch
from transformers import pipeline
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
!huggingface-cli login --token  replace_this_to_huggingface_token

2024-09-21 14:20:07.257301: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 14:20:07.257353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 14:20:07.258734: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
base_model = "google/gemma-2-2b"
# PEFT models paths
peft_models_path = {
    'hindi': "Hemanth-thunder/model_gemma2b_mt-hi_1k_steps",
    'tamil': "Hemanth-thunder/model_gemma2b_mt-ta_2k_steps",
    'kannada': "Hemanth-thunder/model_gemma2b_mt-kh_10k_steps",
    'malayalam': "Hemanth-thunder/model_gemma2b_mt-ml_10k_steps",
    'telugu': "Hemanth-thunder/model_gemma2b_mt-te_10k_steps",
    'bengali': "Hemanth-thunder/model_gemma2b_mt-bn_10k_steps",
    'marathi': "Hemanth-thunder/model_gemma2b_mt-mr_10k_steps",
    'gujarati': "Hemanth-thunder/model_gemma2b_mt-gujarati_gu_10k_steps",
    'odia': "Hemanth-thunder/model_gemma2b_mt-odia_or_10k_steps"
}

# Adapter names
adapter_names = list(peft_models_path.keys())

pipeline_type = "text-generation"

alpaca_prompt = """
### Instruction:
Your are an AI translator to translate the following English input sentence to {} sentence.

### Input:
{}

### Response:
{}"""


In [6]:
## function for generatining text 
def generate(lang, english_text, response_text, merged_model, to_set_lang):
    # Prepare the prompt using the language and texts
    prompt = alpaca_prompt.format(lang.capitalize(), english_text, response_text)

    # Set the adapter for the merged model if specified
    if to_set_lang:
        merged_model.set_adapter(lang)
        print(f"adaptor switched to {lang}")

    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate output using the model
    try:
        outputs = merged_model.generate(**inputs, max_new_tokens=256,do_sample=True,top_p=0.95,temperature=0.2,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id,)
        # Decode and return the response
        response = tokenizer.decode(outputs[0])
        return response.split("### Response:")[-1].strip()
    
    except Exception as e:
        print(f"Error during generation: {e}")
        return None

In [7]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_quant_type="nf4",bnb_4bit_compute_dtype=torch.bfloat16,bnb_4bit_use_double_quant=True,)
model = AutoModelForCausalLM.from_pretrained(base_model, device_map={"": 0},quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
### stack other language with the peft model
#### merging tamil translation with base model with the adapter name tamil

peft_model = PeftModel.from_pretrained(model, peft_models_path['tamil'], adapter_name="tamil")

for lang, path in peft_models_path.items():
    if lang != "tamil":
        try:
            _ = peft_model.load_adapter(path, adapter_name=lang)
            print(f"load {lang}")
        except Exception as e:
            print(f"Error loading adapter for {lang}: {e}")
# Set the model to evaluation mode
peft_model.eval()

In [10]:
%%time 
### testing # lang,english_text,response_text,merged_model
text = "Heavy rain has caused widespread flooding in several parts of the country."

for lang in adapter_names:
    inference_result = generate(lang,text,"",peft_model,to_set_lang=True)
    print(f"inference_result for {lang} : {inference_result}")

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.


adaptor switched to hindi


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


inference_result for hindi : गंभीर वर्षा ने देश के कई भागों में बाढ़ की स्थिति उत्पन्न कर दी है।<eos>
adaptor switched to tamil
inference_result for tamil : நாட்டின் பலப் பகுதிகளில் கனமழை பெய்துள்ளது, அதன் விளைவாக ஏராளமான இடங்களிலும் வெள்ளம் ஏற்பட்டுவிட்டது.
<eos>
adaptor switched to kannada
inference_result for kannada : ದೇಶಾದ್ಯಂತ ಮಳೆಗೆ ತೀವ್ರ ಪ್ರಭಾವ ಬಂದಿದೆ, ಹಲವು ಜಿಲ್ಲೆಯಲ್ಲಿ ಅಪಾಯಕಾರಿ ಸ್ಥಿತಿಯುಂಟಾಗಿದ್ದಾರೆ.<eos>
adaptor switched to malayalam
inference_result for malayalam : രാജ്യത്തെ പലയിടങ്ങളിലും കനത്ത മഴയുടെ ഭീഷണി ഉണ്ടാക്കുന്നുണ്ട്.<eos>
adaptor switched to telugu
inference_result for telugu : చాలా ప్రాంతాల్లో భారీ వర్షం కారణంగా చుట్టూ నీళ్ళతో నిండిపోయింది.
<eos>
adaptor switched to bengali
inference_result for bengali : কয়েকটি অঞ্চলে বৃষ্টিপাতের ফলস্বরূপ প্রায় সব জেলায় পানিধারা দেখা দিয়েছে।<eos>
adaptor switched to marathi
inference_result for marathi : अनेक ठिकाणी पावसाच्या सततधारामुळे पूरस्थिती निर्माण झाली आहे.<eos>
adaptor switched to gujarati
inference_result for gujarati : દેશના 